### 練習問題2.43
Louis Reasonerは練習問題2.42を解くのにだいぶ苦労している。  
彼が書いたqueens⼿続きは動いてはいるようだが、  
⾮常に遅いのだ(6×6の場合でも、Louisは待ちきれなくなっ てしまう)。  
LouisがEva Lu Atorに⾒てくれるよう頼むと、  
彼⼥はLouisがflatmap内のマップのネストの順番を次のように逆にしてしまっていることを指摘した。 

    (flatmap (lambda (new-row)
                (map (lambda (rest-of-queens)
                            (adjoin-position new-row k rest-of-queens))
                     (queen-cols (- k 1)))
                     )
             (enumerate-interval 1 board-size)
             )  

逆にするとなぜプログラムの実⾏が遅くなるのか説明せよ。  
練習問題2.42のプログラムが8クイーンパズルを解く時間をTとして、   
Louisのプログラムがパズルを解くのにかかる時間を⾒積もれ。

チェスの駒の動きについては以下を参照。

http://www.jca-chess.com/chess-2.htm

In [1]:
; 問題文のqueens手続き
; 以下の定義が必要
; ・empty-board
; ・safe?手続き
; ・adjoin-position手続き
(define (queens board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
          (flatmap (lambda (rest-of-queens)
                    (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
                         (enumerate-interval 1 board-size))
                    )
                  (queen-cols (- k 1))
          )
         )
        )
    )
  (queen-cols board-size)
  )

(define (flatmap proc seq)
  (accumulate append '() (map proc seq)))

; 整数列の列挙
(define (enumerate-interval low high)
  (if (> low high) '()
      (cons low (enumerate-interval (+ low 1) high))))

; 集積
(define (accumulate op initial sequence)
  (if (null? sequence) initial
      (op (car sequence) (accumulate op initial (cdr sequence)))))

; フィルタリング
(define (filter predicate sequence)
  (cond ((null? sequence) '())
        ((predicate (car sequence))(cons (car sequence) (filter predicate (cdr sequence))))
        (else (filter predicate (cdr sequence))))
  )

; 回答1
(define empty-board ())

; 回答2
; adjoin-position手続きの呼び出し方が決まっているため、
; この回答しかないはず。
; 列ごとの位置(Y座標)をリストで表す。
(define (adjoin-position new-row k rest-of-queens)
  (append rest-of-queens (list new-row))
  )

; k列目の位置のy座標を返す
(define (get-k-y k positions)
  (define (iter count pos)
    (if (>= count k) (car pos)
        (iter (+ count 1) (cdr pos))
        )
    )
  (iter 1 positions)
  )

; 位置情報のダンプ
(define (print-pos positions)
  (if (null? positions) ()
      (begin
       (display (car positions))
       (display " ")
       (print-pos (cdr positions))
       )
      )
  )

; 回答3
(define (safe? k positions)
  (begin
   ;(display "'")
   ;(print-pos positions)
   (display positions)
   (newline)
   (let ((k-y-pos (get-k-y k positions)))
    (define (iter count pos)
      (if (>= count k) #t
        (if (null? pos) #t
            (let ((y (car pos)))
              (let ((y-upper (- y (- k count)))
                    (y-lower (+ y (- k count))))
                ; y座標が同じか、対角線上にあればFalseを返す。
                ; それ以外はチェックを継続する。
                (if (or (= k-y-pos y) (= k-y-pos y-upper) (= k-y-pos y-lower)) #f
                    (iter (+ count 1) (cdr pos))
                    )
                )
              )
            )
        )
      )
      (iter 1 positions)
    )
   )
  )

;(safe? 4 (list 1 2 3 4))
;(safe? 3 (list 2 4 1 3))
(safe? 4 (list 2 4 1 3))

(2 4 1 3)


#t

In [2]:
(define (queens2 board-size)
  (define (queen-cols k)
    (if (= k 0) (list empty-board)
        (filter
         (lambda (positions) (safe? k positions))
          ;(flatmap (lambda (rest-of-queens)
          ;          (map (lambda (new-row) (adjoin-position new-row k rest-of-queens))
          ;               (enumerate-interval 1 board-size))
          ;          )
          ;        (queen-cols (- k 1))
          ;)
         (flatmap (lambda (new-row)
                    (map (lambda (rest-of-queens)
                           (adjoin-position new-row k rest-of-queens))
                         (queen-cols (- k 1)))
                    )
                  (enumerate-interval 1 board-size)
                  )  
         )
        )
    )
  (queen-cols board-size)
  )


In [3]:
(define (loop max proc n)
  (define (loop-iter count)
    (if (< count max)(begin
                  ;(display (proc n))
                  ;(newline)
                  (proc n)
                  (loop-iter (+ count 1))
        )
    )
   )
  (loop-iter 0)
  )

In [4]:
%%time

(loop 1 queens 4)

(1)
(2)
(3)
(4)
(1 1)
(1 2)
(1 3)
(1 4)
(2 1)
(2 2)
(2 3)
(2 4)
(3 1)
(3 2)
(3 3)
(3 4)
(4 1)
(4 2)
(4 3)
(4 4)
(1 3 1)
(1 3 2)
(1 3 3)
(1 3 4)
(1 4 1)
(1 4 2)
(1 4 3)
(1 4 4)
(2 4 1)
(2 4 2)
(2 4 3)
(2 4 4)
(3 1 1)
(3 1 2)
(3 1 3)
(3 1 4)
(4 1 1)
(4 1 2)
(4 1 3)
(4 1 4)
(4 2 1)
(4 2 2)
(4 2 3)
(4 2 4)
(1 4 2 1)
(1 4 2 2)
(1 4 2 3)
(1 4 2 4)
(2 4 1 1)
(2 4 1 2)
(2 4 1 3)
(2 4 1 4)
(3 1 4 1)
(3 1 4 2)
(3 1 4 3)
(3 1 4 4)
(4 1 3 1)
(4 1 3 2)
(4 1 3 3)
(4 1 3 4)
Time: 1.072159767150879 seconds.



#f

In [5]:
%%time

(loop 1 queens 5)

(1)
(2)
(3)
(4)
(5)
(1 1)
(1 2)
(1 3)
(1 4)
(1 5)
(2 1)
(2 2)
(2 3)
(2 4)
(2 5)
(3 1)
(3 2)
(3 3)
(3 4)
(3 5)
(4 1)
(4 2)
(4 3)
(4 4)
(4 5)
(5 1)
(5 2)
(5 3)
(5 4)
(5 5)
(1 3 1)
(1 3 2)
(1 3 3)
(1 3 4)
(1 3 5)
(1 4 1)
(1 4 2)
(1 4 3)
(1 4 4)
(1 4 5)
(1 5 1)
(1 5 2)
(1 5 3)
(1 5 4)
(1 5 5)
(2 4 1)
(2 4 2)
(2 4 3)
(2 4 4)
(2 4 5)
(2 5 1)
(2 5 2)
(2 5 3)
(2 5 4)
(2 5 5)
(3 1 1)
(3 1 2)
(3 1 3)
(3 1 4)
(3 1 5)
(3 5 1)
(3 5 2)
(3 5 3)
(3 5 4)
(3 5 5)
(4 1 1)
(4 1 2)
(4 1 3)
(4 1 4)
(4 1 5)
(4 2 1)
(4 2 2)
(4 2 3)
(4 2 4)
(4 2 5)
(5 1 1)
(5 1 2)
(5 1 3)
(5 1 4)
(5 1 5)
(5 2 1)
(5 2 2)
(5 2 3)
(5 2 4)
(5 2 5)
(5 3 1)
(5 3 2)
(5 3 3)
(5 3 4)
(5 3 5)
(1 3 5 1)
(1 3 5 2)
(1 3 5 3)
(1 3 5 4)
(1 3 5 5)
(1 4 2 1)
(1 4 2 2)
(1 4 2 3)
(1 4 2 4)
(1 4 2 5)
(1 5 2 1)
(1 5 2 2)
(1 5 2 3)
(1 5 2 4)
(1 5 2 5)
(2 4 1 1)
(2 4 1 2)
(2 4 1 3)
(2 4 1 4)
(2 4 1 5)
(2 5 1 1)
(2 5 1 2)
(2 5 1 3)
(2 5 1 4)
(2 5 1 5)
(2 5 3 1)
(2 5 3 2)
(2 5 3 3)
(2 5 3 4)
(2 5 3 5)
(3 1 4 1)
(3 1 4 2)
(3 1 4 3)
(3 1 4 4)
(3 1 4 5)


#f

In [6]:
%%time

(loop 1 queens2 4)

(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(3 1 1)
(4 1 1)
(4 2 1)
(1 3 1)
(1 4 1)
(2 4 1)
(3 1 2)
(4 1 2)
(4 2 2)
(1 3 2)
(1 4 2)
(2 4 2)
(3 1 3)
(4 1 3)
(4 2 3)
(1 3 3)
(1 4 3)
(2 4 3)
(3 1 4)
(4 1 4)
(4 2 4)
(1 3 4)
(1 4 4)
(2 4 4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1)
(2)
(3)
(4)
(1 1)
(2 1)
(3 1)
(4 1)
(1 2)
(2 2)
(3 2)
(4 2)
(1 3)
(2 3)
(3 3)
(4 3)
(1 4)
(2 4)
(3 4)
(4 4)
(1)
(2)


#f

In [7]:
%%time

(loop 1 queens2 5)

(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1 1)
(2 1)
(3 1)
(4 1)
(5 1)
(1 2)
(2 2)
(3 2)
(4 2)
(5 2)
(1 3)
(2 3)
(3 3)
(4 3)
(5 3)
(1 4)
(2 4)
(3 4)
(4 4)
(5 4)
(1 5)
(2 5)
(3 5)
(4 5)
(5 5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1 1)
(2 1)
(3 1)
(4 1)
(5 1)
(1 2)
(2 2)
(3 2)
(4 2)
(5 2)
(1 3)
(2 3)
(3 3)
(4 3)
(5 3)
(1 4)
(2 4)
(3 4)
(4 4)
(5 4)
(1 5)
(2 5)
(3 5)
(4 5)
(5 5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1 1)
(2 1)
(3 1)
(4 1)
(5 1)
(1 2)
(2 2)
(3 2)
(4 2)
(5 2)
(1 3)
(2 3)
(3 3)
(4 3)
(5 3)
(1 4)
(2 4)
(3 4)
(4 4)
(5 4)
(1 5)
(2 5)
(3 5)
(4 5)
(5 5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1)
(2)
(3)
(4)
(5)
(1 1)
(2 1)
(3 1)
(4 1)
(5 1)
(1 2)
(2 2)
(3 2)
(4 2)
(5 2)
(1 3)
(2 3)
(3 3)
(4 3)
(5 3)
(1 4)
(2 4)
(3 4)
(4 4)
(5 4)
(1 5)
(2 5)
(3 5)
(4 5)
(5 5)


#f

In [8]:
%%time

(loop 1 queens 8)

(1)
(2)
(3)
(4)
(5)
(6)
(7)
(8)
(1 1)
(1 2)
(1 3)
(1 4)
(1 5)
(1 6)
(1 7)
(1 8)
(2 1)
(2 2)
(2 3)
(2 4)
(2 5)
(2 6)
(2 7)
(2 8)
(3 1)
(3 2)
(3 3)
(3 4)
(3 5)
(3 6)
(3 7)
(3 8)
(4 1)
(4 2)
(4 3)
(4 4)
(4 5)
(4 6)
(4 7)
(4 8)
(5 1)
(5 2)
(5 3)
(5 4)
(5 5)
(5 6)
(5 7)
(5 8)
(6 1)
(6 2)
(6 3)
(6 4)
(6 5)
(6 6)
(6 7)
(6 8)
(7 1)
(7 2)
(7 3)
(7 4)
(7 5)
(7 6)
(7 7)
(7 8)
(8 1)
(8 2)
(8 3)
(8 4)
(8 5)
(8 6)
(8 7)
(8 8)
(1 3 1)
(1 3 2)
(1 3 3)
(1 3 4)
(1 3 5)
(1 3 6)
(1 3 7)
(1 3 8)
(1 4 1)
(1 4 2)
(1 4 3)
(1 4 4)
(1 4 5)
(1 4 6)
(1 4 7)
(1 4 8)
(1 5 1)
(1 5 2)
(1 5 3)
(1 5 4)
(1 5 5)
(1 5 6)
(1 5 7)
(1 5 8)
(1 6 1)
(1 6 2)
(1 6 3)
(1 6 4)
(1 6 5)
(1 6 6)
(1 6 7)
(1 6 8)
(1 7 1)
(1 7 2)
(1 7 3)
(1 7 4)
(1 7 5)
(1 7 6)
(1 7 7)
(1 7 8)
(1 8 1)
(1 8 2)
(1 8 3)
(1 8 4)
(1 8 5)
(1 8 6)
(1 8 7)
(1 8 8)
(2 4 1)
(2 4 2)
(2 4 3)
(2 4 4)
(2 4 5)
(2 4 6)
(2 4 7)
(2 4 8)
(2 5 1)
(2 5 2)
(2 5 3)
(2 5 4)
(2 5 5)
(2 5 6)
(2 5 7)
(2 5 8)
(2 6 1)
(2 6 2)
(2 6 3)
(2 6 4)
(2 6 5)
(2 6 6)
(2 6 7)
(2 6 8)
(2 7 1)


#f